In [1]:
from google.colab import drive
drive.mount('/gdrive')
import os

os.chdir('/gdrive/MyDrive/')


Mounted at /gdrive


In [2]:
!pip install hebo==0.3.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 11.1 MB/s 
     |████████████████████████████████| 76.6 MB 1.3 MB/s 
     |████████████████████████████████| 2.6 MB 66.8 MB/s 
     |████████████████████████████████| 245 kB 77.8 MB/s 
     |████████████████████████████████| 959 kB 48.5 MB/s 
     |████████████████████████████████| 71 kB 11.4 MB/s 
     |████████████████████████████████| 155 kB 64.4 MB/s 
     |████████████████████████████████| 249 kB 78.3 MB/s 
     |████████████████████████████████| 72 kB 1.5 MB/s 
     |████████████████████████████████| 207 kB 76.8 MB/s 
  Created wheel for GPy: filename=GPy-1.10.0-cp38-cp38-linux_x86_64.whl size=2783623 sha256=dbab4f0a30c3c82f80b7774ecfcaa04efc5cf04ef2fb05df93a2ee333d17b320
  Stored in directory: /root/.cache/pip/wheels/48/b3/22/31f07cfd7b182ea3703151b7e5a7d6447e3e1ac6aa5c529413
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-a

In [3]:
!pip install pymoo==0.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.7 MB 29.2 MB/s 
     |████████████████████████████████| 239 kB 76.4 MB/s 
  Attempting uninstall: cma
    Found existing installation: cma 3.2.2
    Uninstalling cma-3.2.2:
      Successfully uninstalled cma-3.2.2
  Attempting uninstall: pymoo
    Found existing installation: pymoo 0.6.0.1
    Uninstalling pymoo-0.6.0.1:
      Successfully uninstalled pymoo-0.6.0.1


In [4]:
import pandas as pd
import numpy  as np
from hebo.design_space.design_space import DesignSpace
from hebo.optimizers.hebo import HEBO

def obj(params : pd.DataFrame) -> np.ndarray:
    return ((params.values - 0.37)**2).sum(axis = 1).reshape(-1, 1)
        
space = DesignSpace().parse([{'name' : 'x', 'type' : 'num', 'lb' : -3, 'ub' : 3}])
opt   = HEBO(space)
for i in range(5):
    rec = opt.suggest(n_suggestions = 1)
    opt.observe(rec, obj(rec))
    print(rec)
    print(obj(rec))
    print('After %d iterations, best obj is %.2f' % (i, opt.y.min()))

     x
0 -3.0
[[11.3569]]
After 0 iterations, best obj is 11.36
     x
0  0.0
[[0.1369]]
After 1 iterations, best obj is 0.14
          x
14  2.95233
[[6.66842726]]
After 2 iterations, best obj is 0.14
         x
2  0.00245
[[0.13509326]]
After 3 iterations, best obj is 0.14
           x
24 -0.418307
[[0.62142812]]
After 4 iterations, best obj is 0.14


In [5]:
!pip install GPy
!pip install sklearn
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=2dc1be8498bb2c8944cb67788cb7d748c848eba197cdbfb3dd3c772a7541cae9
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 8.3 MB/s 


In [6]:
#The 3 following cells aim at simuling the behavior of NN models with a simple model for MNIST
!pip install ray==1.2.0
!pip install -U hyperopt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import torch.nn.functional as F
import math
import numpy as np
import random
import math 
from ray import tune
from hyperopt import hp, fmin, tpe, Trials
from functools import *
from ray.tune.logger import *
import time

EPOCH_SIZE = 32*32*8*32
TEST_SIZE = 256*32*32 #remove 1024

#This is a function that can be used by several NN model
def train(model, optimizer ,func ,train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    #for (data, target) in train_loader:
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
           # print("hehe")
            return
        # We set this just for the example to run quickly.
        data = np.repeat(data, 3, 1)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
       # print(output)
       # print(F.log_softmax(output, dim=1))
       # print(target)
        loss = func(output, target)
        loss.backward()
        optimizer.step()
        
#This is a function that can be used by several NN model (it only does accuracy ATM)
def test(model, func, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0.
    total = 0.
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            data = np.repeat(data, 3, 1)

            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)

            total += target.size(0)
            correct += (predicted == target).sum().item()


                
    return correct / total


torch.set_num_threads(8)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47.3 MB 1.9 MB/s 
     |████████████████████████████████| 236 kB 70.3 MB/s 
     |████████████████████████████████| 90 kB 10.7 MB/s 
     |████████████████████████████████| 71 kB 11.1 MB/s 
     |████████████████████████████████| 128 kB 80.7 MB/s 
     |████████████████████████████████| 3.0 MB 63.8 MB/s 
     |████████████████████████████████| 201 kB 85.0 MB/s 
     |████████████████████████████████| 280 kB 60.5 MB/s 
     |████████████████████████████████| 58 kB 2.9 MB/s 
  Created wheel for gpustat: filename=gpustat-1.0.0-py3-none-any.whl size=19887 sha256=55bb8c8ee992c67716e2673d1200ccec9dc82f76e406180b360e1259c20cbb97
  Stored in directory: /root/.cache/pip/wheels/1b/ed/14/0d513c962b25da841c42022cb5847c2ef835902c8563b8fb01
Successfully built gpustat
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 34.1 MB/s 
     |████████████████████████████████| 200 kB 69.1 MB/s 
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.1.2
    Uninstalling hyperopt-0.1.2:
      Successfully uninstalled hyperopt-0.1.2


In [7]:
# A random mnist from the internet to get a correct model to reason about

class train_mnist():
    def __init__(self,config):
        
        self.config = {
        "sigmoid_func": 1
      ,  "hidden_dim":64
      ,  "n_layer":3    }
        for key, value in config.items():
            self.config[key] = value
        config = self.config
        
        self.i = 0
        
       # mnist_transforms = transforms.Compose(
       #     [transforms.ToTensor(),
       #      transforms.Normalize((0.1307, ), (0.3081, ))])
        mnist_transforms = transforms.ToTensor()

        self.train_loader = DataLoader(
            datasets.FashionMNIST(path, train=True, download=True , transform=mnist_transforms),
            batch_size=1024,
            shuffle=True)
       # self.test_loader = DataLoader(
       #     datasets.MNIST("/gdrive/MyDrive", train=False, transform=mnist_transforms),
       #     batch_size=64,
       #     shuffle=True)

        
        test_valid_dataset = datasets.FashionMNIST(path, train=False, transform=mnist_transforms)
        valid_ratio = 0.5  
        nb_test = int((1.0 - valid_ratio) * len(test_valid_dataset))
        nb_valid =  int(valid_ratio * len(test_valid_dataset))
        test_dataset, val_dataset = torch.utils.data.dataset.random_split(test_valid_dataset, [nb_test, nb_valid])
        self.test_loader =  DataLoader(test_dataset,
            batch_size=1024,
            shuffle=True)

        self.val_loader =  DataLoader(val_dataset,
            batch_size=1024,
            shuffle=True)

        sigmoid_func_uniq = nn.Tanh()

        from torchvision import models

        self.model = models.resnet50(num_classes = 30) #LeNet(192,64,10,
                    #3,
                    #config.get("droupout_prob",0.5) ,sigmoid_func_uniq)
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.get("lr", 0.01),  
                                     amsgrad=True)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
    
    def adapt(self, config):
        #print(self.optimizer)
        temp = copy.deepcopy(self)
        for key, value in config.items():
            temp.config[key] = value
        config = temp.config

       # temp.model.adapt(config.get("droupout_prob", 0.5))
        temp.optimizer = torch.optim.Adam(temp.model.parameters(), lr=config.get("lr", 0.01), 
                                     amsgrad=True)
        return temp
    
    
# All NN models should have a function train1 and test1 that calls the common train and test defined above.
# train1 and test1 is then used in the scheduler
    def train1(self):
        print("iteration: " + str(self.i) )
        self.i+=1
        train(self.model, self.optimizer, F.nll_loss, self.train_loader)

    def val1(self):
        return test(self.model, F.nll_loss, self.val_loader)
        
    def test1(self):
        return test(self.model, F.nll_loss, self.test_loader)

    def step(self):
        train1()
        return val1()

# __INCEPTION_SCORE_begin__
class LeNet(nn.Module):
    """
    LeNet for MNist classification, used for inception_score
    """

    def __init__(self,input_dim, hidden_dim, output_dim, n_layers,
            drop_prob, sigmoid ):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(drop_prob)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def adapt(self,drop_prob):
        self.conv2_drop = nn.Dropout2d(drop_prob)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)



# Convolution Neural network using Pytorch 
class ConvNet(nn.Module):
    def __init__(self,input_dim, hidden_dim, output_dim, n_layers,
                 drop_prob, sigmoid ):
        super(ConvNet, self).__init__()
        
        self.sigmoid = sigmoid
        self.i_d = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)

        self.fc = nn.Linear(input_dim, output_dim)
        self.first= nn.Linear(input_dim, hidden_dim)
        self.hidden = [nn.Linear(hidden_dim,hidden_dim) for _ in range(self.n_layers)]
        self.drop_out = nn.Dropout(drop_prob)

        self.last = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.sigmoid(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, self.i_d)
        x=self.first(x)
        x=self.drop_out(x)
        for i in range(self.n_layers):
            x=self.hidden[i](x)
            x=self.drop_out(x)
        x = self.last(x)
        return F.log_softmax(x, dim=1)


#class Resnet(nn.Module):
#  def __init__(self,config):
#    super(ConvNet, self).__init__()
#    self.under = torchvision.models.resnet50()

#  def adapt(self, config):



In [8]:
path = "/gdrive/MyDrive/"


In [9]:
class Parent():
    """Parent Class that handles the passage of Network Configurations from one step to the
    following
    """
    def __init__(self, point_hyperspace, configuration, model, loss):
        self.point_hyperspace = point_hyperspace
        self.configuration_list = [configuration]
        self.loss_list = [np.array(loss)]
        self.model = model
        self.is_replicated = False

    def update(self, configuration, loss, model):
        self.is_replicated = False
        self.configuration_list.append(configuration)
        self.loss_list=np.append(self.loss_list,loss)
        self.model = model

    def replication(self, n_children):
        self.is_replicated = True
      #  self.configuration_list.append(self.configuration_list[-1])
      #  self.loss_list=np.append(self.loss_list,self.loss_list[-1])
    #    replication_trials(self.point_hyperspace.trials, n_children)

    def get_last_conf(self):
        return self.configuration_list[-1]

    def get_point_hyperspace(self):
        return self.point_hyperspace

    def get_model(self):
        return self.model

    def get_loss(self):
        return self.loss_list

In [10]:
!pip install scikit-optimize
import copy

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as F
import torch

import math
import numpy as np
import random
import math

from hyperopt import hp, fmin,atpe ,tpe, Trials

from functools import *
#import skipy-optimize as skopt 
from skopt import gp_minimize
from skopt.plots import plot_objective


class GPGuesser():
    """Used to sample the hyperspace with the tools of `hyperopt`
    """

    def __init__(self, searchspace, verbose):
        self.searchspace = searchspace
        self.string = "aiteration"

        self.verbose = verbose
        self.algo = partial(tpe.suggest, n_startup_jobs=1)



    def repeat_good(self, trials, iteration, function, configuration):
        space = copy.deepcopy(self.searchspace)

     #   curr_eval = getattr(trials, '_ids')
     #   if curr_eval == set():
     #       curr_eval = 0
     #   else:
     #       curr_eval = max(curr_eval) + 1

       # space[self.string] = iteration #hp.quniform(self.string, -.5+iteration, .5+iteration, 1)
        space = []
        for k,v in configuration.items():
            space.append(v)
        loss=function(space)
        space.append((iteration))

        trials[0].append(space)
        trials[1] = np.append(trials[1],loss)


    #    print(curr_eval)
     #   coucou = skopt.gp_minimize(function,space,n_calls=1,x0=trials[0],y0=trials[1],n_initial_points=0)
        
        
      #  trials[0] = coucou.x_iters
      #  trials[1] = coucou.func_vals
      # # fmin(
       #     function,
       #     space, algo=self.algo,
       #     max_evals=curr_eval+nb_eval,
       #     trials=trials,
       #     verbose=self.verbose
       # )
       # _ = plot_objective(coucou)
        
        
        
        
        
    #    space = copy.deepcopy(configuration)

    #    for k, v in configuration.items():
    #        space[k] = hp.uniform(k, v - 1e-10, v + 1e-10)

    #    curr_eval = getattr(trials, '_ids')

    #    if curr_eval == set():
    #        curr_eval = 0
    #    else:
    #        curr_eval = max(curr_eval) + 1

     #   space[self.string] = iteration # hp.quniform(self.string, -.5+iteration, .5+iteration, 1)
        #print(space)
     #   fmin(
     #       function,
     #       space,
     ##       algo=self.algo,
      #      max_evals=curr_eval+1,
       #     trials=trials,
       #     verbose=self.verbose
       # )

    def compute_once(self, trials, iteration, function):
        space = copy.deepcopy(self.searchspace)

        curr_eval = getattr(trials, '_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) + 1

        space[self.string] = iteration # hp.quniform(self.string, -.5+iteration, .5+iteration, 1)
        fmin(
            function,
            space,
            algo=self.algo,
            max_evals=curr_eval+1,
            trials=trials,
            verbose=self.verbose
        )

    def compute_batch(self, trials, nb_eval, iteration, function):
        space = copy.deepcopy(self.searchspace)

     #   curr_eval = getattr(trials, '_ids')
     #   if curr_eval == set():
     #       curr_eval = 0
     #   else:
     #       curr_eval = max(curr_eval) + 1

       # space[self.string] = iteration #hp.quniform(self.string, -.5+iteration, .5+iteration, 1)
        space.append((0,iteration+1e-10))
      
    #    print(curr_eval)
        temp=0
        if isinstance(trials[0],type(None)):
            temp =2
        
        coucou = gp_minimize(function,space,n_calls=nb_eval,x0=trials[0],y0=trials[1],n_initial_points=temp)
        
        for i in coucou.x_iters:
            i[-1] = iteration
        trials[0] = coucou.x_iters
        
        trials[1] = coucou.func_vals
       # fmin(
       #     function,
       #     space, algo=self.algo,
       #     max_evals=curr_eval+nb_eval,
       #     trials=trials,
       #     verbose=self.verbose
       # )
        if(iteration==10):
            
            _ = plot_objective(coucou)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 9.6 MB/s 


In [11]:
def translation(liste):
   # config = {}
  #  config["lr"] = liste[0]
    #config["droupout_prob"] = liste[1]
    #config["weight_decay"] = liste[2]
    #config["b1"] = liste[3]
    #config["b2"] = liste[4]
    return liste#config
def test_function(x,models,h,losses, parent_model,k_f,iteration):
    x= translation(x)

    if (isinstance(k_f,list)):
            k=k_f[0]
            Islist = True 
    else:
            k = k_f
            Islist = False
    print(k)
    if iteration == 0:
        models[k] = parent_model[k](x)
    else:      
        models[k] = parent_model.adapt(x)
    if(Islist):
        k_f[0] += 1
    
    #for key, value in x.items():
    #        print(key + " "+str(x[key]))

    h[k] = x
    #start_time = time.time()
    models[k].train1()
    loss = models[k].test1()
    test = models[k].val1()
    #print("--- %s seconds ---" % (time.time() - start_time))

    temp = dict(x)
    temp.update({'loss' : loss})
    temp.update({'test' : test})
    fsvnlogger.on_result(temp)

    losses[k] = -loss
    print("accuracy, " + str(loss) + "\n")
    return -loss

In [12]:

# Oracle (Paul) TODO
path = ""
class Oracle():
    def __init__(self, searchspace ):
        #self.hyperspace is the original (input) searchspace
        self.searchspace = searchspace

    def repeat_good(self,trials, iteration,function,configuration): #add space
        space = copy.deepcopy(configuration)
        for k,v in configuration.items():
            space[k] =  hp.uniform(k,-1e-10+v,v + 1e-10) 

        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+1, trials=trials)
        
    def compute_once(self,trials, iteration,function): #add space

        space = copy.deepcopy(self.searchspace)
        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+1, trials=trials)
        
        
    def compute_batch(self,trials, nb_eval, iteration,function): #add space

        space = copy.deepcopy(self.searchspace)
        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
            
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+nb_eval, trials=trials)
    


In [13]:
class Guesser():
    """Used to sample the hyperspace with the tools of `hyperopt`
    """

    def __init__(self, searchspace, verbose):
        self.searchspace = searchspace
        self.string = "aiteration"

        self.verbose = verbose
        self.algo = partial(tpe.suggest, n_startup_jobs=1)



    def repeat_good(self, trials, iteration, function, configuration):

        space = copy.deepcopy(configuration)

        for k, v in configuration.items():
            space[k] = hp.uniform(k, v - 1e-10, v + 1e-10)

        curr_eval = getattr(trials, '_ids')

        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) + 1

        space[self.string] = iteration # hp.quniform(self.string, -.5+iteration, .5+iteration, 1)
        #print(space)
        fmin(
            function,
            space,
            algo=self.algo,
            max_evals=curr_eval+1,
            trials=trials,
            verbose=self.verbose
        )

    def compute_once(self, trials, iteration, function):
        space = copy.deepcopy(self.searchspace)

        curr_eval = getattr(trials, '_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) + 1

        space[self.string] = iteration # hp.quniform(self.string, -.5+iteration, .5+iteration, 1)
        fmin(
            function,
            space,
            algo=self.algo,
            max_evals=curr_eval+1,
            trials=trials,
            verbose=self.verbose
        )

    def compute_batch(self, trials, nb_eval, iteration, function):
        space = copy.deepcopy(self.searchspace)
        print(space)
        curr_eval = getattr(trials, '_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) + 1

        space[self.string] = iteration #hp.quniform(self.string, -.5+iteration, .5+iteration, 1)
        print(curr_eval)

        fmin(
            function,
            space, algo=self.algo,
            max_evals=curr_eval+nb_eval,
            trials=trials,
            verbose=self.verbose
        )



In [14]:
def set_iteration(algo,iteration):
  algo.space.paras["aiteration"].lb=iteration
  algo.space.paras["aiteration"].ub=iteration

class Guesser():
    """Used to sample the hyperspace with the tools of `hyperopt`
    """

    def __init__(self, searchspace, verbose):
        self.searchspace = searchspace
        self.string = "aiteration"

        self.verbose = verbose
        print(self.searchspace)
        self.algo = HEBO(searchspace)


    def repeat_good(self, trials, iteration, function, configuration):

        configuration = copy.deepcopy(configuration)
        configuration["aiteration"] = iteration
        print(configuration)
        rec = pd.DataFrame(configuration,index=[0])   
        res = np.array([np.array([function(configuration)])])
        self.algo.observe(rec,res)

    def compute_batch(self, trials, nb_eval, iteration, function):
        set_iteration(self.algo, iteration) 
        for i in range(nb_eval):
         rec = self.algo.suggest(n_suggestions = 1,fix_input = {"aiteration":iteration})
         rec1 = rec.to_dict()
         for key in rec1:
          rec1[key] = rec1[key][list(rec1[key].keys())[0]] 
         res = np.array([np.array([function(rec1)])])
         self.algo.observe(rec,res)



In [ ]:
import copy


class Scheduler():
    def __init__(self, model, num_iteration, num_config,
                 oracle):
        #Oracle manages the Bayesian optimization
        self.oracle = oracle
        self.iteration = num_iteration
        self.num_config = num_config 
        self.sqrt_config = 2 # math.floor(math.sqrt(num_config)) # math.ceil(num_config/5) #
        self.n_parents = 1#self.sqrt_config
        #self.h is for the m "h" used at every loop, h is a configuration from the search space
        self.h = np.repeat({},num_config) 
        
        #self.out is for storing the result of the algorithm, ie all "h" from all iterations
        #from all sqrt(m) best models per iterations.
        self.out = np.zeros((num_iteration,self.sqrt_config))
        
        #self.hyperspaces is for storing the sqrt(m) hyperspaces used by the algorithm
        self.hyperspaces = np.zeros(self.sqrt_config)
        
        self.plot = np.zeros(num_iteration)

        
        #self.model is the m model that will explore new hyperspace points at every iterations
        self.models = np.repeat(model,num_config)
        
        #self.parents is the sqrt(m) best model from last iteration
        self.parents = np.repeat(model,self.sqrt_config)

        #self.losses remembers the performances of all m models at one iteration to decide which ones are the sqrt(m) best from self.models.
        self.losses = np.zeros(num_config)
        
        self.k = [0] # c'est pour avoir un pointeur sur k, c'est pas plus que O(sqrt)-paralélisable  pour le moment du coup.
    
    def initialisation(self):
        num_config = self.num_config
        extended_Hyperspace = Trials() #[None,None]
        fmin_objective = partial(test_function, models=self.models,h=self.h,losses=self.losses,parent_model=self.models, k_f = self.k,iteration = 0)
        self.oracle.compute_batch(extended_Hyperspace ,num_config , 0 ,fmin_objective)
            
        indexes = np.argsort(self.losses)     
        self.out[0] = (self.losses[indexes])[0:self.sqrt_config]
        self.hyperspaces = np.repeat(extended_Hyperspace,self.sqrt_config)    
        self.parents = np.array([Parent(copy.deepcopy(extended_Hyperspace),(self.h[indexes])[i], (self.models[indexes])[i],(self.losses[indexes])[i])  
                                 for i in range(self.sqrt_config) ])         
        self.plot[0] = self.losses[indexes][0]
        
    def loop(self):
        sqrt_config = self.sqrt_config
        iteration = self.iteration
        for i in range(1,iteration):
            
            self.k[0] = 0
            
            start_time = time.time()
            for j in range(self.n_parents):
                parent = self.parents[j]
                point_extended_hyperspace = parent.get_point_hyperspace()
                print("\n loss of parent " + str(parent.get_loss()[-1]) )
                print("\n loss " + str(parent.get_loss()))
                
                fmin_objective = partial(test_function, models=self.models,h=self.h,losses=self.losses,parent_model=parent.get_model(), k_f = self.k,iteration = len(parent.get_loss()))

                    
                if not parent.is_replicated:
                    print('not replicated')
                    self.oracle.repeat_good(
                        point_extended_hyperspace,
                        len(parent.get_loss()),
                        fmin_objective,
                        parent.configuration_list[-1]
                    )

                    # computes the new batch for each one of the parents for every iteration
                    self.oracle.compute_batch(
                        point_extended_hyperspace,
                        int(self.num_config/self.n_parents) - 1,
                        len(parent.get_loss()),
                        fmin_objective
                    )
                else:

                    print('replicated')
                    self.oracle.compute_batch(
                        point_extended_hyperspace,
                        int(self.num_config/self.n_parents),
                        len(parent.get_loss()),
                        fmin_objective
                    )
                    

            #self.oracle.Repeat_good(extended_Hyperspace ,i ,fmin_objective,parent.configuration_list[-1])
             #   self.oracle.compute_Batch(extended_Hyperspace ,int(self.num_config/sqrt_config) -1 , i ,fmin_objective)
           
            print("totalt time: " +  str(time.time() - start_time))


            combined_losses = np.concatenate(
                        (
                            self.losses,
                            
                                [self.parents[i].get_loss()[-1].item() for i in range(self.n_parents)]
                            
                        ),
                        0
                    )
            ixs_parents = np.argsort(combined_losses)
            parent_idx = ixs_parents[:self.n_parents]
            print(combined_losses)
            print(parent_idx)
            # ??? why saving it in a numpt array ?
            # It is creating the new Parent `array`
            temp_parents = [''] * self.n_parents

            for j, x in enumerate(parent_idx):
                # ??? why converting it to integer ?
                x = int(x)
                if x >= self.num_config:
                    temp_parents[j] = copy.deepcopy(self.parents[x - self.num_config])
                    temp_parents[j].replication(self.n_parents)
                else:
                    temp_parents[j] = copy.deepcopy(self.parents[math.floor(x/self.num_config * self.n_parents)])
                    temp_parents[j].update(self.h[x], self.losses[x], self.models[x])
               #     temp_parents[j].point_hyperspace = Trials()

            self.parents = temp_parents


class FSVNLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "FSNV_MNIST_GP.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()

            #if not self._continuing:
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   

config= {
     "lr": hp.loguniform("lr",-10*2.3,0)

    #    , "droupout_prob": hp.uniform("droupout_prob",0,1)
#          ,   "weight_decay": hp.loguniform("weight_decay",-5*2.3,-1*2),
#    "b1" : 1-hp.loguniform("b1",-4*2.3, -1*2.3),
#    "b2" : 1-hp.loguniform("b2",-5*2.3, -2*2.3)
}


config = DesignSpace().parse([{'name' : 'lr', 'type' : 'pow', 'lb' : math.e**-10, 'ub' : 1},
                              {'name' : 'aiteration', 'type' : 'int', 'lb' : 0, 'ub' : 0}])

fsvnlogger = FSVNLogger(config,"")




    
CONFIGURATION = 4
ITERATIONS = 10

model = train_mnist
oracle = Guesser(config,0)


scheduler = Scheduler(
  model,
  ITERATIONS,
  CONFIGURATION,
  oracle) 

start_time = time.time()
scheduler.initialisation()     
scheduler.loop()     
print("totalt time: " +  str(time.time() - start_time))


0
iteration: 0
accuracy, 0.4058

1
iteration: 0
accuracy, 0.1064

2
iteration: 0


In [ ]:
for _ in range(5):


  scheduler = Scheduler(
    model,
    ITERATIONS,
    CONFIGURATION,
    oracle) 

  start_time = time.time()
  scheduler.initialisation()     
  scheduler.loop()     
  print("totalt time: " +  str(time.time() - start_time))



In [ ]:
for _ in range(5):


  scheduler = Scheduler(
    model,
    ITERATIONS,
    CONFIGURATION,
    oracle) 

  start_time = time.time()
  scheduler.initialisation()     
  scheduler.loop()     
  print("totalt time: " +  str(time.time() - start_time))



In [ ]:
for _ in range(10):
  oracle = BayesOpt(config)
  start_time = time.time()
  oracle.compute_Once(function)
  print("totalt time: " +  str(time.time() - start_time))

In [ ]:
#Normal Random Opt
evals = 4
crypt = np.zeros(evals)
def function(x):
    print(x)
    model = train_mnist(x)
    for _ in range(20):
        model.train1()
        loss = model.test1()
        test = model.val1()
        temp = dict(x)
        temp.update({'loss' : loss})
        temp.update({'test' : test})

        temp.update({'iteration' :  model.i})
        hyperlogger.on_result(temp)
    
  #  loss = model.test1()
    return -loss

In [ ]:
class RandomOpt():
    def __init__(self, searchspace ):
        self.searchspace = searchspace

    def compute_Once(self,function): 
        fmin(function, self.searchspace, algo=partial(tpe.rand.suggest), max_evals=evals, trials=Trials())
    
config= {
     "lr": hp.uniform("lr",0,.1),
     "droupout_prob": hp.uniform("droupout_prob",0,1),
     "weight_decay": hp.uniform("weight_decay",0,.1),
    "b1" : hp.uniform("b1",0.9, 1),
    "b2" : hp.uniform("b2",0.99, 1)
    
}
oracle = RandomOpt(config)

In [ ]:
class RandomLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "trash.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()

            #if not self._continuing:
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   
hyperlogger = RandomLogger(config,"")

start_time = time.time()
oracle.compute_Once(function)
print("totalt time: " +  str(time.time() - start_time))


In [ ]:
class BayesOpt():
    def __init__(self, searchspace ):
        self.searchspace = searchspace

    def compute_Once(self,function): 
        fmin(function, self.searchspace, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=evals, trials=Trials())
    
config= {
     "lr": hp.loguniform("lr",-5*2.3,-1*2)
    , "droupout_prob": hp.uniform("droupout_prob",0,1)
          ,   "weight_decay": hp.loguniform("weight_decay",-5*2.3,-1*2),
    "b1" : 1-hp.loguniform("b1",-4*2.3, -1*2.3),
    "b2" : 1-hp.loguniform("b2",-5*2.3, -2*2.3)
}
oracle = BayesOpt(config)

In [ ]:
class HyperLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "hyperoptFMNIST.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()

            #if not self._continuing:
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   
hyperlogger = HyperLogger(config,"")

  


In [ ]:
# A random mnist from the internet to get a correct model to reason about
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
#from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import json
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import torch
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn

import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler 
from ray.tune.schedulers import MedianStoppingRule

from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
#from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
#from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import torch
#import adabelief_pytorch
global_checkpoint_period=np.inf
from ray.tune.schedulers.pb2 import PB2

class train_mnist_pb2(tune.Trainable):
    def setup(self, config):
        self.config = {
        "sigmoid_func": 1
      ,  "hidden_dim":43
      ,  "n_layer":2    }
        for key, value in config.items():
            self.config[key] = value
        config = self.config
        
        self.i = 0
        
        #mnist_transforms = transforms.Compose(
        #    [transforms.ToTensor(),
        #     transforms.Normalize((0.1307, ), (0.3081, ))])
        mnist_transforms = transforms.ToTensor()

        self.train_loader = DataLoader(
            datasets.FashionMNIST("/gdrive/MyDrive", train=True, download=True, transform=mnist_transforms),
            batch_size=1024,
            shuffle=True)
       # self.test_loader = DataLoader(
       #     datasets.MNIST("~/data", train=False, transform=mnist_transforms),
       #     batch_size=64,
       #     shuffle=True)

        
        test_valid_dataset = datasets.FashionMNIST("/gdrive/MyDrive", train=False, transform=mnist_transforms)
        valid_ratio = 0.5  
        nb_test = int((1.0 - valid_ratio) * len(test_valid_dataset))
        nb_valid =  int(valid_ratio * len(test_valid_dataset))
        test_dataset, val_dataset = torch.utils.data.dataset.random_split(test_valid_dataset, [nb_test, nb_valid])
        self.test_loader =  DataLoader(test_dataset,
            batch_size=1024,
            shuffle=True)

        self.val_loader =  DataLoader(val_dataset,
            batch_size=1024,
            shuffle=True)

        sigmoid_func_uniq = nn.Tanh()


        self.model = LeNet(192,64,10,
                    3,
                    config.get("droupout_prob",0.5) ,sigmoid_func_uniq)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        if config.get("b1", 0.999)>=1:
          temp = 1 - 1e-10
        else:
          temp = 1-config.get("b1", 0.999)
                
        if config.get("b2", 0.999)>=1:
          temp1 = 1 - 1e-10
        else:
          temp1 = 1-config.get("b2", 0.999)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((temp,temp1)),
                                     eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)

    

    
    def reset_config(self, config):
        if "lr" in config:
            for param_group in self.optimizer.param_groups:
                param_group["lr"] = config.get("lr", 0.01)
        if "b1" in config:
            for param_group in self.optimizer.param_groups:
                param_group["betas"] = (
                    (1-1e-10 if config.get("b1", 0.999)>=1 else 1-config.get("b1", 0.999),(1-1e-10 if config.get("b2", 0.999)>=1 else 1-config.get("b2", 0.999))))
        if "wd" in config:
            for param_group in self.optimizer.param_groups:
                param_group["weight_decay"] = weight_decay
        
        self.model.adapt(config.get("droupout_prob", 0.5))
        self.config = config
        return True
    
# All NN models should have a function train1 and test1 that calls the common train and test defined above.
# train1 and test1 is then used in the scheduler
    def train1(self):
        print("iteration: " + str(self.i) )
        self.i+=1
        train(self.model, self.optimizer, F.nll_loss, self.train_loader)

    def val1(self):
        return test(self.model, F.nll_loss, self.val_loader)
        
    def test1(self):
        return test(self.model, F.nll_loss, self.test_loader)

    def step(self):
        self.train1()
        return {'loss' : self.test1(), 'test' : self.val1()}
    
    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save({
            "model": self.model.state_dict(),
            "optim": self.optimizer.state_dict(),

        }, path)

        return checkpoint_dir

    def load_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint["model"])
        self.optimizer.load_state_dict(checkpoint["optim"])
epsilon = 1e-10



algo = HyperOptSearch(metric="loss",
    mode="max")
algo = ConcurrencyLimiter(algo, max_concurrent=25)
scheduler = PB2(
    time_attr="training_iteration",
    perturbation_interval=1,
    hyperparam_bounds={
        # distribution for resampling
     "lr": [1e-5,1e-1] #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":[1e-5,1e-1]#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": [1e-4, 1e-1]#,1e-4), #*10 et 0
 ,    "b2": [1e-5, 1e-2] #,1e-4), #*10 et 0
 ,    "droupout_prob": [0+epsilon, 1-epsilon]#,1e-4), #*10 et 0
    }) 

imageSize = 32


class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "trash.csv") #aller jusqu'a 9
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   

cst=imageSize
total=cst*cst*3
def trial_name_id(trial):
    return f"{trial.trainable_name}_{trial.trial_id}"
from ray import tune


for _ in range(1):
  ray.shutdown()
  ray.init()
  start_time = time.time()
  analysis = tune.run(
      
  train_mnist_pb2,
  scheduler=scheduler,
  reuse_actors=True,
  search_alg=algo,
  verbose=2,
  checkpoint_at_end=True,
  num_samples=4,
  # export_formats=[ExportFormat.MODEL],
config= {
     "lr": tune.loguniform(1e-5,1e-1)
    , "droupout_prob": tune.uniform(0,1)
          ,   "weight_decay": tune.loguniform(1e-5,1e-1),
    "b1" : tune.loguniform(1e-4, 1e-1),
    "b2" : tune.loguniform(1e-5, 1e-2)
},      stop={
          "training_iteration": 20,
      },        metric="loss",
      mode="max"
,resources_per_trial={'cpu':2 ,'gpu': 1}
              ,     loggers=[TestLogger])
  print("time "+ str(time.time()- start_time))

In [ ]:
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import *
class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "BOHB_FMNISTvalid.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()

config= {
     "lr": tune.loguniform(1e-5,1e-1)
    , "droupout_prob": tune.uniform(0,1)
          ,   "weight_decay": tune.loguniform(1e-5,1e-1),
    "b1" : tune.loguniform(1e-4, 1e-1),
    "b2" : tune.loguniform(1e-5, 1e-2)
}

for i in range(10):
  class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "BOHB_FMNISTvalid"+str(i)+".csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
    ray.shutdown()
    ray.init()
    start_time = time.time()
    algo = TuneBOHB(metric="loss", mode="max")
    bohb = HyperBandForBOHB(
        time_attr="training_iteration",
        metric="loss",
        mode="max",
        max_t=20)
    analysis = tune.run(train_mnist_pb2, config=config, scheduler=bohb, search_alg=algo,num_samples=4,loggers=[TestLogger],resources_per_trial={'cpu':2 ,'gpu': 1})

    print("time "+ str(time.time()- start_time))

In [ ]:
!pip install ConfigSpace
!pip install hpbandster


In [ ]:


import argparse

import ray
from ray import tune
from ray.tune.schedulers.pb2 import PB2
from ray.tune.schedulers import PopulationBasedTraining

from ray.tune.examples.pbt_function import pbt_function


for i in range(9,11):
    class TestLogger(tune.logger.Logger):
        def _init(self):
            progress_file = os.path.join(path, "PBT_FMNIST"+str(i)+".csv")
            self._continuing = os.path.exists(progress_file)
            self._file = open(progress_file, "a")
            self._csv_out = None
        def on_result(self, result):
            tmp = result.copy()
            result = flatten_dict(tmp, delimiter="/")
            if self._csv_out is None:
                self._csv_out = csv.DictWriter(self._file, result.keys())
                self._csv_out.writeheader()
            self._csv_out.writerow(
                {k: v
                 for k, v in result.items() if k in self._csv_out.fieldnames})
            self._file.flush()

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    args, _ = parser.parse_known_args()
    ray.shutdown()
    ray.init()  # force pausing to happen for test

    epsilon = 1e-10
    from ray.tune.suggest.hyperopt import HyperOptSearch
    from ray.tune.suggest import ConcurrencyLimiter

    algo = HyperOptSearch(metric="loss",
        mode="max")
    algo = ConcurrencyLimiter(algo, max_concurrent=4)

    pbt = PopulationBasedTraining(
        perturbation_interval=1,
            time_attr="training_iteration",

        hyperparam_mutations={
            # hyperparameter bounds.
     "lr": [1e-5,1e-1] #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":[1e-5,1e-1]#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": [1e-4, 1e-1]#,1e-4), #*10 et 0
 ,    "b2": [1e-5, 1e-2] #,1e-4), #*10 et 0
 ,    "droupout_prob": [0+epsilon, 1-epsilon]#,1e-4), #*10 et 0
        })

    analysis = tune.run(
        train_mnist_pb2,
  checkpoint_at_end=True,
        scheduler=pbt,
        metric="loss",
        mode="max",
        search_alg = algo,
        verbose=0,
        stop={
            "training_iteration": 20,
        },
        num_samples=4,
  reuse_actors=True,
loggers=[TestLogger],
        
config= {
     "lr": tune.loguniform(1e-5,1e-1)
    , "droupout_prob": tune.uniform(0,1)
          ,   "weight_decay": tune.loguniform(1e-5,1e-1),
    "b1" : tune.loguniform(1e-4, 1e-1),
    "b2" : tune.loguniform(1e-5, 1e-2)
}
                      ,resources_per_trial={'cpu':2 ,'gpu': 1})

    print("Best hyperparameters found were: ", analysis.best_config)

In [ ]:
from ray.tune.schedulers.pb2_utils import normalize, optimize_acq, \
            select_length, UCB, standardize, TV_SquaredExp

In [ ]:


import argparse

import ray
from ray import tune
from ray.tune.schedulers.pb2 import PB2
from ray.tune.schedulers import PopulationBasedTraining

from ray.tune.examples.pbt_function import pbt_function


for i in range(1):
    class TestLogger(tune.logger.Logger):
        def _init(self):
            progress_file = os.path.join(path, "PBT_FMNIST_CVPR"+str(i)+".csv")
            self._continuing = os.path.exists(progress_file)
            self._file = open(progress_file, "a")
            self._csv_out = None
        def on_result(self, result):
            tmp = result.copy()
            result = flatten_dict(tmp, delimiter="/")
            if self._csv_out is None:
                self._csv_out = csv.DictWriter(self._file, result.keys())
                self._csv_out.writeheader()
            self._csv_out.writerow(
                {k: v
                 for k, v in result.items() if k in self._csv_out.fieldnames})
            self._file.flush()

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    args, _ = parser.parse_known_args()
    ray.shutdown()
    ray.init()  # force pausing to happen for test

    epsilon = 1e-10
    from ray.tune.suggest.hyperopt import HyperOptSearch
    from ray.tune.suggest import ConcurrencyLimiter

    algo = HyperOptSearch(metric="loss",
        mode="max")
    algo = ConcurrencyLimiter(algo, max_concurrent=4)

    pbt = PopulationBasedTraining(
        perturbation_interval=1,
            time_attr="training_iteration",

        hyperparam_mutations={
            # hyperparameter bounds.
     "lr": [1e-10,1] #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10

        })

    analysis = tune.run(
        train_mnist_pb2,
  checkpoint_at_end=True,
        scheduler=pbt,
        metric="loss",
        mode="max",
        search_alg = algo,
        verbose=0,
        stop={
            "training_iteration": 10,
        },
        num_samples=4,
  reuse_actors=True,
loggers=[TestLogger],
        
config= {
     "lr": tune.loguniform(1e-10,1)
}
                      ,resources_per_trial={'cpu':2 ,'gpu': 1})

    print("Best hyperparameters found were: ", analysis.best_config)

In [ ]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)